In [1]:
import pandas as pd
import numpy as np

In [103]:
dic = {
    'pitstop_time' : 30,
    'laps' : 80,
    'pits' : 4,
    'top_cap' : 39,
    'bottom_cap' : 5,
}


df = pd.DataFrame(dic, index=[0])
# write df to csv
df.to_csv('configs.csv', index=False)
df

,pitstop_time,laps,pits,top_cap,bottom_cap
0,30,80,4,39,5


In [2]:

class TireStrategy : 
    def __init__(self) :
        self.S_laps = 0
        self.M_laps = 0
        self.H_laps = 0
        self.time = 0
        self.pitstops = 0

    def __str__(self) :
        return 'Soft: ' + str(self.S_laps) + ' Medium: ' + str(self.M_laps) + ' Hard: ' + str(self.H_laps) + ' Time: ' + str(self.time) + ' Pitstops: ' + str(self.pitstops)


def get_all_options_lap_capped(laps, pits, options, bottom_cap=5, top_cap=44, current_sum=0, current_combination=None):
    if current_combination is None:
        current_combination = []

    if current_sum == laps and len(current_combination) == pits and all(num <= top_cap for num in current_combination) and all(num >= bottom_cap for num in current_combination):
        options.append(current_combination)
    elif current_sum < laps and len(current_combination) < pits:
        for i in range(1, laps + 1):
            if current_sum + i <= laps and i <= top_cap and i >= bottom_cap:  # Check if adding i exceeds the target laps and if i is less than or equal to 44
                get_all_options_lap_capped(laps, pits, options, bottom_cap, top_cap, current_sum + i, current_combination + [i])

                

def get_all_options(laps, pits, options, current_sum=0, current_combination=None):
    if current_combination is None:
        current_combination = []

    if current_sum == laps and len(current_combination) == pits :
        options.append(current_combination)
    elif current_sum < laps and len(current_combination) < pits:
        for i in range(1, laps + 1):
            if current_sum + i <= laps :  # Check if adding i exceeds the target laps and if i is less than or equal to 44
                get_all_options(laps, pits, options, current_sum + i, current_combination + [i])


def soft(x):
    return 118 + np.exp(0.33 * x - 2)
def medium(x):
    return 120 + np.exp(0.31 * x - 6.1)
def hard(x):
    return 122 + np.exp(0.3 * x - 9.4)

def get_best_tire_option (laps):
    result = TireStrategy()
    S = cumulative_soft(laps)
    M = cumulative_medium(laps)
    H = cumulative_hard(laps)
    
    if S < M and S < H : 
        result.S_laps = laps
        result.time = S
    elif M < S and M < H :
        result.M_laps = laps
        result.time = M
    else :
        result.H_laps = laps
        result.time = H
    return result



def cumulative_soft(x):
    soft_values = [soft(i) for i in range(1, x+1)]
    return sum(soft_values)

def cumulative_medium(x):
    medium_values = [medium(i) for i in range(1, x+1)]
    return sum(medium_values)

def cumulative_hard(x):
    hard_values = [hard(i) for i in range(1, x+1)]
    return sum(hard_values)

def cumulative_soft1(x):
    sum = 0
    for i in range(1, x+1):
        sum += soft(i)
    return sum

def cumulative_medium1(x):
    sum = 0
    for i in range(1, x+1):
        sum += medium(i)
    return sum

def cumulative_hard1(x):
    sum = 0
    for i in range(1, x+1):
        sum += hard(i)
    return sum

def toPrettyTime(sek) :
    return str(int(sek / 60)) + ':' + str(int(sek % 60))

def readCsv() : 
    df = pd.read_csv('configs.csv')
    return df





In [12]:
import time
start_time = time.time()
def find_pit_combinations(laps, pits):
    def generate_combinations(current_combination, remaining_laps, remaining_pits):
        if remaining_pits == 0:
            if remaining_laps == 0:
                combinations.append(sorted(current_combination.copy()))
            return

        for i in range(1, remaining_laps + 1):
            current_combination.append(i)
            generate_combinations(current_combination, remaining_laps - i, remaining_pits - 1)
            current_combination.pop()

    combinations = []
    generate_combinations([], laps, pits)
    return np.unique(np.array(combinations), axis=0)

laps = 60
pits = 5
result = find_pit_combinations(laps, pits)
end_time = time.time()

print(f'time: {end_time - start_time}')
print(len(result))

time: 2.2243072986602783
5260


In [15]:
import numpy as np
from tqdm import tqdm

import time
start_time = time.time()
def find_pit_combinations(laps, pits, bottom_cap=1, top_cap=float('inf')):
    def generate_combinations(current_combination, remaining_laps, remaining_pits, bottom, top):
        if remaining_pits == 0:
            if remaining_laps == 0:
                combinations.append(sorted(current_combination.copy()))
            return
            
        for i in range(bottom, min(remaining_laps, top) + 1):
            current_combination.append(i)
            generate_combinations(current_combination, remaining_laps - i, remaining_pits - 1, i, top)
            current_combination.pop()

    combinations = []
    generate_combinations([], laps, pits, bottom_cap, top_cap)
    return np.unique(np.array(combinations), axis=0)

laps = 100
pits = 6
bottom_cap = 5  # Set your bottom_cap value
top_cap = 38    # Set your top_cap value
result = find_pit_combinations(laps, pits, bottom_cap, top_cap)


end_time = time.time()

print(f'time: {end_time - start_time}')
print(len(result))


time: 0.19806241989135742
27501


In [16]:
pitstop_time = 0
laps = 0
pits = 0
top_cap = 0
bottom_cap = 0
options = None
strategies = []


loadconifg = input('Load config? (y/n) ')
loadconifg = loadconifg.lower()
if loadconifg == 'y' :
    df = readCsv()
    laps = int(df['laps'])
    pits = int(df['pits'])
    if pits == 0 : pits = 0
    else : pits += 1
    top_cap = int(df['top_cap'])
    bottom_cap = int(df['bottom_cap'])
    pitstop_time = int(df['pitstop_time'])
else :
    laps = int(input('Laps: '))
    pits = int(input('Pits: '))
    if pits == 0 : pits = 0
    else : pits += 1
    set_caps = input('set caps? (y/n) ')
    set_caps = set_caps.lower()
    if set_caps == 'y' :
        bottom_cap = int(input('Bottom cap: '))
        top_cap = int(input('Top cap: '))
    else :
        bottom_cap = 5
        top_cap = 40
        print(f'Using default caps {bottom_cap} and {top_cap}')


print('your configurations:')
print(f'Laps: {laps}')
print(f'Pits: {pits}')
print(f'Bottom cap: {bottom_cap}')
print(f'Top cap: {top_cap}')
print(f'Pitstop time: {pitstop_time}')
print('-----------------------------------------------------------\n')
start_time = time.time()



# -----------------------------------------
# option calclation
# -----------------------------------------
print('now calculating options...')
if(laps // pits <= bottom_cap or laps // pits >= top_cap) :
    print('Laps per stint not capped')
    options = find_pit_combinations(laps, pits, bottom_cap, top_cap)
else :
    print(f'Laps per stint capped at {bottom_cap} and {top_cap}')
    options = find_pit_combinations(laps, pits, bottom_cap, top_cap)

time_to_calculate_options = time.time()
print('-----------------------------------------------------------')
print('calculated ' + str(len(options)) + ' options')
print(f'in {round(time_to_calculate_options-start_time, 3)} seconds ')
print('-----------------------------------------------------------\n')

# -----------------------------------------
# option elimination
# -----------------------------------------
time_to_eliminat_duplicates = any
if(len(options) > 150000) :
    print('to many options to calculate all strategies => eliminating duplicates')
    print('this will take a while...')
    no_duplicates = []
    for option in tqdm(options, desc="Eliminating duplicates", unit="option"):
        sorted_option = sorted(option)
        if sorted_option not in no_duplicates:
            no_duplicates.append(sorted_option)

    options = no_duplicates
    time_to_eliminat_duplicates = time.time()
    print(f'Time to eliminate duplicates {round(time_to_eliminat_duplicates-time_to_calculate_options, 3)} seconds\n')
else :
    time_to_eliminat_duplicates = time.time()
    print('eliminating duplicates not necessary\n')

# -----------------------------------------
# calculating Strategies
# ----------------------------------------- 
print('-----------------------------------------------------------')
print('now calculating Strategies...')
for option in tqdm(options, desc="Processing Options", unit="option"):
    strat = TireStrategy()
    stints = []

    for stint in range(0, len(option)) :
        stints.append(get_best_tire_option(option[stint]))

    for stint in range(0, len(stints)) :
        if(stint != 0) :
            strat.time += 25
            strat.pitstops += 1
        strat.S_laps += stints[stint].S_laps
        strat.M_laps += stints[stint].M_laps
        strat.H_laps += stints[stint].H_laps
        strat.time += stints[stint].time
    strategies.append(strat)

time_to_calculate_strategy = time.time()
print(f'Time to calculate all Strategies {round(time_to_calculate_strategy-time_to_eliminat_duplicates, 3)} seconds ')
print('-----------------------------------------------------------\n')    


# -----------------------------------------
# getting best Strategy
# ----------------------------------------- 
print('***********************************************************') 
print(f'total time: {round(time_to_calculate_strategy-start_time, 3)} seconds ')
print('Best Stratgy:')
best = TireStrategy()
besttime = 9999999999999999999999

for strategy in strategies :
    if strategy.time < besttime :
        best = strategy
        besttime = strategy.time
c
print('S: ' + str(best.S_laps) + ' M: ' + str(best.M_laps) + ' H: ' + str(best.H_laps) + ' Time: ' + toPrettyTime(best.time) + ' Pitstops: ' + str(best.pitstops))
print("When to Pit:")
best_option = options[strategies.index(best)]
print(str(best_option))
print('***********************************************************') 

your configurations:
Laps: 100
Pits: 7
Bottom cap: 6
Top cap: 37
Pitstop time: 0
-----------------------------------------------------------

now calculating options...
Laps per stint capped at 6 and 37
-----------------------------------------------------------
calculated 407254 options
in 3.286 seconds 
-----------------------------------------------------------

to many options to calculate all strategies => eliminating duplicates
this will take a while...


Eliminating duplicates:  13%|█▎        | 52297/407254 [01:14<08:28, 698.34option/s] 


KeyboardInterrupt: 

In [44]:
import numpy as np
laps = 40
pitstops = laps // 10
strategies = []




for pits in range(0, pitstops) :
    print('Pitstops: ' + str(pits))
    arr = np.ones(pits, dtype=int)
    print(arr)
    strategy = TireStrategy()
    if(arr == []) : 
        strategies.append(get_best_tire_option(laps))
    else :
        for stint in len(arr) :
            strategy.time += pitstop_time # pitstop
            for i in range(1, laps+1) :
                arr[stint]  
            
           



IndentationError: expected an indented block (738803066.py, line 19)

In [47]:
laps = 8

res = get_best_tire_option(laps)
print(toPrettyTime(res.time))
print("with tires: " + str(res.S_laps) + " soft, " + str(res.M_laps) + " medium, " + str(res.H_laps) + " hard) ")

15:50
with tires: 8 soft, 0 medium, 0 hard) 


In [34]:
import numpy as np
laps = 39
onestop = 0
nostop = 0


for i in range (0, laps+1) :
    nostop += hard(i)

for i in range (0, int(laps/2)+1) :
    onestop += hard(i)
for i in range (0, int(laps/2)+1) :
    onestop += hard(i)
onestop += 35

print('No stop: ' + toPrettyTime(nostop))
print('One stop: ' + toPrettyTime(onestop))


No stop: 81:58
One stop: 81:55


In [31]:
import numpy as np

laps = int(input('Laps: '))
times_S_M = []
hards_limit = 36 # mit integral = 39

hards = (laps // hards_limit)*hards_limit
if(laps >= hards_limit) :
    laps = laps % hards_limit



for i in range(0, laps+1):
    strat = TireStrategy()
    for stint in range(0, laps-i):
        strat.time += soft(stint)
        strat.S_laps += 1

    if(i == 0 or i == laps):
        pass
    else :
        strat.time += 25

    for stint in range(0, i):
        strat.time += medium(stint)
        strat.M_laps += 1

    times_S_M.append(strat)



best_strategy = min(times_S_M, key=lambda x: x.time)
best_strategy.H_laps = hards
print('Best strategy: ' + toPrettyTime(best_strategy.time) + 's')
print(f'Tires: {best_strategy.S_laps} Soft, {best_strategy.M_laps} Medium, {best_strategy.H_laps} Hard')

Best strategy: 1:58s
Tires: 1 Soft, 0 Medium, 36 Hard


## Mäßig gutes erstes beispiel

In [33]:
import numpy as np
times_S_M = []
times_S_H = []
laps = 50
max_pits = laps/10


for i in range(0, laps+1):
    strat = TireStrategy()
    for stint in range(0, laps-i):
        strat.time += soft(stint)
        strat.S_laps += 1

    if(i == 0 or i == laps):
        pass
    else :
        strat.time += 25

    for stint in range(0, i):
        strat.time += medium(stint)
        strat.M_laps += 1

    times_S_M.append(strat)



for i in range(0, laps+1):
    strat = TireStrategy()
    for stint in range(0, laps-i):
        strat.time += soft(stint)
        strat.S_laps += 1

    if(i == 0 or i == laps):
        pass
    else :
        strat.time += 25

    for stint in range(0, i):
        strat.time += hard(stint)
        strat.H_laps += 1

    times_S_H.append(strat)

best_strategy = min(times_S_M, key=lambda x: x.time)
print('Best strategy: ' + toPrettyTime(best_strategy.time) + 's')
print(f'Tires: {best_strategy.S_laps} Soft, {best_strategy.M_laps} Medium, {best_strategy.H_laps} Hard')


best_strategy = min(times_S_H, key=lambda x: x.time)
print('Best strategy: ' + toPrettyTime(best_strategy.time) + 's')
print(f'Tires: {best_strategy.S_laps} Soft, {best_strategy.M_laps} Medium, {best_strategy.H_laps} Hard')


Best strategy: 104:5s
Tires: 18 Soft, 32 Medium, 0 Hard
Best strategy: 101:53s
Tires: 13 Soft, 0 Medium, 37 Hard
